Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
%%time

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_size = 1024
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  relu = tf.nn.relu(hidden_biases)
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  def net(ds):
    hidden_layer = tf.matmul(ds, hidden_weights) + hidden_biases
    relu_layer = tf.nn.relu(hidden_layer)
    return tf.matmul(relu_layer, weights) + biases
  
  # Training computation.
  logits = net(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))    
  test_prediction = tf.nn.softmax(net(tf_test_dataset))



num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.53 µs
Initialized
Minibatch loss at step 0: 305.731110
Minibatch accuracy: 12.5%
Validation accuracy: 29.9%
Minibatch loss at step 500: 24.186081
Minibatch accuracy: 84.4%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 9.285927
Minibatch accuracy: 74.2%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 5.053465
Minibatch accuracy: 82.0%
Validation accuracy: 81.9%


In [21]:
%%time

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_size = 1024
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  relu = tf.nn.relu(hidden_biases)
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  def net(ds):
    hidden_layer = tf.matmul(ds, hidden_weights) + hidden_biases
    relu_layer = tf.nn.relu(hidden_layer)
    o = tf.matmul(relu_layer, weights) + biases
    return o
    return (o +
      beta*(
      tf.nn.l2_loss(hidden_weights) +
      tf.nn.l2_loss(weights)))
  
  # Training computation.
  logits = tf.nn.dropout(net(tf_train_dataset), 0.5)
  beta = 0.01
  reg = beta*(tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))    
  test_prediction = tf.nn.softmax(net(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
# W:B:R:W:B:L2    90.2, 90.0

Initialized
Minibatch loss at step 0: 3539.242920
Minibatch accuracy: 9.4%
Validation accuracy: 19.3%
Minibatch loss at step 500: 21.986282
Minibatch accuracy: 51.6%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 1.672355
Minibatch accuracy: 50.0%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 1.408112
Minibatch accuracy: 46.9%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 1.333085
Minibatch accuracy: 49.2%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 1.349300
Minibatch accuracy: 52.3%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 1.496389
Minibatch accuracy: 50.8%
Validation accuracy: 82.9%
Test accuracy: 89.2%
CPU times: user 2min 4s, sys: 6.03 s, total: 2min 10s
Wall time: 2min 10s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 12

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_size = 1024
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  relu = tf.nn.relu(hidden_biases)
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  def net(ds):
    hidden_layer = tf.matmul(ds, hidden_weights) + hidden_biases
    relu_layer = tf.nn.relu(hidden_layer)
    o = tf.matmul(relu_layer, weights) + biases
    return o
  
  # Training computation.
  logits = net(tf_train_dataset)
  beta = 0.01
  reg = beta*(tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))    
  test_prediction = tf.nn.softmax(net(tf_test_dataset))


In [10]:
%%time
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    sub_step = step%10
    offset = (sub_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
# W:B:R:W:B:L2    90.2, 90.0

Initialized
Minibatch loss at step 0: 3412.172852
Minibatch accuracy: 8.3%
Validation accuracy: 20.6%
Test accuracy: 21.7%
Minibatch loss at step 500: 27.756376
Minibatch accuracy: 100.0%
Validation accuracy: 63.0%
Test accuracy: 69.5%
Minibatch loss at step 1000: 0.430550
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Test accuracy: 76.4%
Minibatch loss at step 1500: 0.204788
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Test accuracy: 76.3%
Minibatch loss at step 2000: 0.189691
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Test accuracy: 76.3%
Minibatch loss at step 2500: 0.181798
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Test accuracy: 76.3%
Minibatch loss at step 3000: 0.175812
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Test accuracy: 76.4%
CPU times: user 39.3 s, sys: 3.13 s, total: 42.4 s
Wall time: 42.5 s


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [18]:
batch_size = 12

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_size = 1024
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
  
  relu = tf.nn.relu(hidden_biases)
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  def net(ds):
    hidden_layer = tf.matmul(ds, hidden_weights) + hidden_biases
    relu_layer = tf.nn.relu(hidden_layer)
    o = tf.matmul(relu_layer, weights) + biases
    return o
  
  # Training computation.
  logits = tf.nn.dropout(net(tf_train_dataset), 0.5)
#   logits = net(tf_train_dataset)
  beta = 0.01
  reg = beta*(tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))    
  test_prediction = tf.nn.softmax(net(tf_test_dataset))


In [19]:
%%time
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    sub_step = step%50
    offset = (sub_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
# W:B:R:W:B:L2    90.2, 90.0

Initialized
Minibatch loss at step 0: 3595.817139
Minibatch accuracy: 0.0%
Validation accuracy: 16.6%
Test accuracy: 17.3%
Minibatch loss at step 500: 22731175231488.000000
Minibatch accuracy: 8.3%
Validation accuracy: 28.4%
Test accuracy: 29.5%
Minibatch loss at step 1000: 3328978372384094420992.000000
Minibatch accuracy: 8.3%
Validation accuracy: 29.1%
Test accuracy: 30.4%
Minibatch loss at step 1500: 369951588088486935455927894016.000000
Minibatch accuracy: 16.7%
Validation accuracy: 21.5%
Test accuracy: 22.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%
CPU times: user 38.8 s, sys: 3.63 s, total: 42.4 s
Wall time: 42.6 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
def try_hpar(batch_size = 128,
             num_steps = 3001,
             hidden_size = 1024,
             droput_keep=0.9,
             learning_rate=0.5,
             decay_rate=.95,
             beta = 0.01):

    graph = tf.Graph()
    with graph.as_default():  
      global_step = tf.Variable(0)

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      
      # Variables.
      hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_size]))
      hidden_biases = tf.Variable(tf.zeros([hidden_size]))

      relu = tf.nn.relu(hidden_biases)

      weights = tf.Variable(
        tf.truncated_normal([hidden_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      def net(ds):
        hidden_layer = tf.matmul(ds, hidden_weights) + hidden_biases
        relu_layer = tf.nn.relu(hidden_layer)
        o = tf.matmul(relu_layer, weights) + biases
        return o

      # Training computation.
      logits = tf.nn.dropout(net(tf_train_dataset), droput_keep)
    #   logits = net(tf_train_dataset)
      
      reg = beta*(tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + reg)

      # Optimizer.
      learning_rate = tf.train.exponential_decay(learning_rate=learning_rate,
                                                 global_step=global_step,
                                                 decay_steps=num_steps,
                                                 decay_rate=decay_rate)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(net(tf_valid_dataset))    
      test_prediction = tf.nn.softmax(net(tf_test_dataset))

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
#       print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
#         if (step % 500 == 0):
#             print("Minibatch loss at step %d: %f" % (step, l))
#             print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#             print("Validation accuracy: %.1f%%" % accuracy(
#         valid_prediction.eval(), valid_labels))
    
      return accuracy(test_prediction.eval(), test_labels)
#       print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [6]:
%%time

from itertools import product

def unpack(x):
    return [(list(x.items())[0][0], v) for v in list(x.items())[0][1]]

def make_train_grid():
    grid_config = {'batch_size': [128],
                   'num_steps': [3001],
                   'hidden_size': [1024],
                   'droput_keep': [0.5],
                   'learning_rate': [0.5],
                   'decay_rate': [0.95, 0.99],
                   'beta': [0.01, 0.005]}
    
    p = product(*[unpack({k:v}) for k,v in grid_config.items()])
    return list([dict(c) for c in p])

train_grid = make_train_grid()

# for hpar in train_grid:
#     try_hpar(**hpar)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 196 µs


In [7]:
print('Will try the following (grid size:', len(train_grid),')')
for hpar in train_grid:
    print(hpar)

Will try the following (grid size: 4 )
{'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.95, 'beta': 0.01}
{'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.95, 'beta': 0.005}
{'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.99, 'beta': 0.01}
{'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.99, 'beta': 0.005}


In [8]:
print('Starting grid seach')
result = []
for hpar in train_grid:
    %time r = try_hpar(**hpar)
    print([r, hpar])
    result.append([r, hpar])
    

Starting grid seach
CPU times: user 1min 59s, sys: 8.25 s, total: 2min 7s
Wall time: 2min 8s
[89.230000000000004, {'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.95, 'beta': 0.01}]
CPU times: user 1min 57s, sys: 9.33 s, total: 2min 6s
Wall time: 2min 6s
[90.370000000000005, {'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.95, 'beta': 0.005}]
CPU times: user 1min 49s, sys: 8.7 s, total: 1min 58s
Wall time: 1min 58s
[89.290000000000006, {'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.99, 'beta': 0.01}]
CPU times: user 1min 52s, sys: 9.1 s, total: 2min 2s
Wall time: 2min 2s
[90.590000000000003, {'num_steps': 3001, 'learning_rate': 0.5, 'batch_size': 128, 'droput_keep': 0.5, 'hidden_size': 1024, 'decay_rate': 0.99, 'beta': 0.005}]


In [9]:
result

[[89.230000000000004,
  {'batch_size': 128,
   'beta': 0.01,
   'decay_rate': 0.95,
   'droput_keep': 0.5,
   'hidden_size': 1024,
   'learning_rate': 0.5,
   'num_steps': 3001}],
 [90.370000000000005,
  {'batch_size': 128,
   'beta': 0.005,
   'decay_rate': 0.95,
   'droput_keep': 0.5,
   'hidden_size': 1024,
   'learning_rate': 0.5,
   'num_steps': 3001}],
 [89.290000000000006,
  {'batch_size': 128,
   'beta': 0.01,
   'decay_rate': 0.99,
   'droput_keep': 0.5,
   'hidden_size': 1024,
   'learning_rate': 0.5,
   'num_steps': 3001}],
 [90.590000000000003,
  {'batch_size': 128,
   'beta': 0.005,
   'decay_rate': 0.99,
   'droput_keep': 0.5,
   'hidden_size': 1024,
   'learning_rate': 0.5,
   'num_steps': 3001}]]